In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_excel("Commodities Daily Realized Covariance (Neural).xlsx", sheet_name= "Vol", index_col="Date",parse_dates=True)

In [3]:
df.head()

,CORN_USD,NATGAS_USD,SOYBN_USD,WTICO_USD,SUGAR_USD,WHEAT_USD,XAU_USD
Date,,,,,,,
2006-04-03,0.014116,0.032091,0.009071,0.013889,0.013271,0.015175,0.007742
2006-04-04,0.009410,0.025314,0.009071,0.013109,0.018067,0.011347,0.009796
2006-04-05,0.010407,0.026743,0.011381,0.016712,0.014813,0.009933,0.010602
2006-04-06,0.011937,0.030397,0.010730,0.013730,0.017031,0.013756,0.010642
2006-04-07,0.012829,0.024131,0.008789,0.013190,0.022469,0.015885,0.011747


In [4]:
train=df.iloc[:int(len(df)*0.7)]

In [5]:
test=df.iloc[int(len(df)*0.7)-22:]

In [6]:
df.iloc[int(len(df)*0.7)]

,2015-11-16
CORN_USD,0.010249
NATGAS_USD,0.031684
SOYBN_USD,0.008225
WTICO_USD,0.020245
SUGAR_USD,0.029853
WHEAT_USD,0.014775
XAU_USD,0.006555


In [7]:
train=np.array(train)

In [8]:
train.shape

(2427, 7)

In [9]:
RV_d = []
for i in range(len(train)-1):
  RV_d.append(train[i].astype(float))

RV_w = []
for i in range(len(train)-5):
  RV_w.append(np.mean(train[i:i+5]).astype(float))

RV_m = []
for i in range(len(train)-22):
  RV_m.append(np.mean(train[i:i+22]).astype(float))

RV_d=RV_d[len(RV_d)-len(RV_m):]
RV_w=RV_w[len(RV_w)-len(RV_m):]

In [10]:
RV_D = np.zeros([len(RV_d),train.shape[1]])
for i in range(len(RV_d)):
  RV_D[i,:]=RV_d[i]

RV_W = np.zeros([len(RV_w),train.shape[1]])
for i in range(len(RV_w)):
  RV_W[i,:]=RV_w[i]

RV_M = np.zeros([len(RV_m),train.shape[1]])
for i in range(len(RV_m)):
  RV_M[i,:]=RV_m[i]

In [11]:
RV_actuals=train[22:,:]

In [12]:
RV_actuals.shape

(2405, 7)

In [13]:
test=np.array(test)

In [14]:
test.shape

(1063, 7)

In [15]:
  for i in range(len(test)-1):
    RV_d.append(test[i].astype(float))

  RV_w = []
  for i in range(len(test)-5):
    RV_w.append(np.mean(test[i:i+5]).astype(float))

  RV_m = []
  for i in range(len(test)-22):
    RV_m.append(np.mean(test[i:i+22]).astype(float))

  RV_d=RV_d[len(RV_d)-len(RV_m):]
  RV_w=RV_w[len(RV_w)-len(RV_m):]
  RV_D_test = np.zeros([len(RV_d),test.shape[1]])
  for i in range(len(RV_d)):
    RV_D_test[i,:]=RV_d[i]

  RV_W_test = np.zeros([len(RV_w),test.shape[1]])
  for i in range(len(RV_w)):
    RV_W_test[i,:]=RV_w[i]

  RV_M_test = np.zeros([len(RV_m),test.shape[1]])
  for i in range(len(RV_m)):
    RV_M_test[i,:]=RV_m[i]

In [16]:
forecasts=np.zeros([test.shape[0]-22,test.shape[1]])
for j in range(test.shape[1]):
  Data ={'Actual': RV_actuals[:,j],
  'D': RV_D[:,j],
  'W': RV_W[:,j],
  'M': RV_M[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  rv_scaled = sm.add_constant(rv_scaled)
  rv_scaled['Actual']=rv_scaled['Actual'].astype(float)
  rv_scaled['D']=rv_scaled['D'].astype(float)
  rv_scaled['W']=rv_scaled['W'].astype(float)
  rv_scaled['M']=rv_scaled['M'].astype(float)
  X = rv_scaled.drop("Actual", axis = 1)
  y = rv_scaled["Actual"]
  results = sm.OLS(y, X).fit()
  Data ={'D': RV_D_test[:,j],
  'W': RV_W_test[:,j],
  'M': RV_M_test[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  X_test = sm.add_constant(rv_scaled)
  forecasts[:,j]= results.predict(X_test)

In [17]:
df1=df.iloc[int(len(df)*0.7):]

In [18]:
df1.head()

,CORN_USD,NATGAS_USD,SOYBN_USD,WTICO_USD,SUGAR_USD,WHEAT_USD,XAU_USD
Date,,,,,,,
2015-11-16,0.010249,0.031684,0.008225,0.020245,0.029853,0.014775,0.006555
2015-11-17,0.009642,0.031884,0.008519,0.022250,0.018494,0.012005,0.010144
2015-11-18,0.009066,0.029299,0.009077,0.022657,0.020383,0.012946,0.007008
2015-11-19,0.009731,0.033841,0.007430,0.030108,0.019616,0.012735,0.008387
2015-11-20,0.011871,0.024932,0.007031,0.027704,0.022391,0.009718,0.009802


In [19]:
for i in range(len(df1.columns)):
  df1[df1.columns[i]]=forecasts[:,i]

<ipython-input-19-8a232fbfd1e1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[df1.columns[i]]=forecasts[:,i]


In [20]:
df1.to_excel("HAR Forecasts.xlsx")

In [21]:
test=df.iloc[int(len(df)*0.7):]
test=np.array(test)
RMSE=np.sqrt(np.sum((test-forecasts)**2)/(test.shape[0]*test.shape[1]))
MAE=(np.sum(abs(test-forecasts))/(test.shape[0]*test.shape[1]))
QLIKE=np.sum(test/forecasts-np.log(test/forecasts)-1)/(test.shape[0]*test.shape[1])
MAPE=np.sum(abs(test-forecasts)/(test))/(test.shape[0]*test.shape[1])
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

RMSE: 0.004388117285128566 | QLIKE: 0.038601044523828454 | MAE: 0.0030930913494282807 | MAPE: 0.2408237801379752
